In [2]:
#@title
!pip install PyDictionary

# Explanation
First we need to install TensorFlow in the machine. Then `import` them.


In [3]:
#@title
import requests
from contextlib import closing
import codecs
import csv
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras import utils
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from PyDictionary import PyDictionary
pydict = PyDictionary()
import urllib.request as urllib

# Reproducability :D 
np.random.seed(7)

randomTextURL = 'https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output0.txt'
keyMashTextURL = 'https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output1.txt'
#bigNotGibberishURL = 'https://raw.githubusercontent.com/rrenaud/Gibberish-Detector/master/big.txt'

Using TensorFlow backend.


We then download training data for the neural network and format it in a way that is readable by the network.

In [0]:
#@title
data_words, data_labels = [], []

with requests.get(randomTextURL) as randomText:
  for row in randomText.text.split('\n'):
    if row is not []:
      row = row.split(',')
      data_words.append(row[0])
      data_labels.append('Random Text')

with requests.get(keyMashTextURL) as keyMash:
  for row in keyMash.text.split('\n'):
    if row is not []:
      row = row.split(',')
      data_words.append(row[0])
      data_labels.append('Keyboard Mash')
  
#for word in bigNotGibberish:
#  pass # something
  
# lets take 80% data as training and remaining 20% for test.
train_size = int(len(data_words) * .8)
 
train_words = data_words[:]#train_size]
train_labels = data_labels[:]#train_size]

test_words = data_words[train_size:]
test_labels = data_labels[train_size:]
  
# 2 different types
num_labels = 2
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_words)
 
x_train = tokenizer.texts_to_matrix(train_words, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_words, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

y_train = utils.to_categorical(y_train, num_classes=num_labels)
y_test = utils.to_categorical(y_test, num_classes=num_labels)

Then make and train the neural network

In [24]:
#@title
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=[vocab_size]))
#model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels, activation='softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                      epochs=5,
                    verbose=1,
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               7680512   
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 1026      
Total params: 7,944,194
Trainable params: 7,944,194
Non-trainable params: 0
_________________________________________________________________
Train on 20655 samples, validate on 2295 samples
Epoch 1/5
20655/20655 [==============================] - 6s 269us/step - loss: 0.5441 - acc: 0.7618 - val_loss: 0.1841 - val_acc: 0.8797
Epoch 2/5
 7200/20655 [=========>....................] - ETA: 3s - loss: 0.1384 - acc: 0.9475

20655/20655 [==============================] - 5s 250us/step - loss: 0.1374 - acc: 0.9470 - val_loss: 0.1974 - val_acc: 0.9294
Epoch 3/5
20655/20655 [==============================] - 5s 251us/step - loss: 0.0820 - acc: 0.9513 - val_loss: 0.2249 - val_acc: 0.9011
Epoch 4/5
20655/20655 [==============================] - 5s 250us/step - loss: 0.0733 - acc: 0.9551 - val_loss: 0.2205 - val_acc: 0.9098
Epoch 5/5
20655/20655 [==============================] - 5s 250us/step - loss: 0.0720 - acc: 0.9552 - val_loss: 0.2353 - val_acc: 0.9176


In [28]:
#@title
inp = input('Enter a term to test: > ')

output = model.predict(tokenizer.texts_to_matrix([inp], mode='tfidf'), batch_size=batch_size)
print(output)
if output[0][0] > 0.75: 
  print('Keyboard Mashing')
elif output[0][1] > 0.75:
  print('Random Text')

Enter a term to test: > ZQOEIEHZUUUKJUO
[[1.9290135e-04 9.9980718e-01]]
Random Text


In [0]:
#@title
# Given a single, one-line string, determine if it contains an english word or
# not. If it does, determine whether this random text was produced via keyboard
# mashing or if it was randomly generated by a computer. Keyboard mashing likely
# contains letters around a certain key: if one key is pressed, as well as 6
# surrounding keys (at least 4), and this pattern appears for 75% of all of the
# random text, then it was probably mashed.

gibberish = True
gibberish_count = 0
total_gib_count = 0

def determine_gibberish(line):
  global gibberish
  global gibberish_count
  global total_gib_count
  words = line.split()
  print(pydict.meaning(line))
  if pydict.meaning(line) is None:
    gibberish_count+=1
    total_gib_count+=1
  else:
    total_gib_count+=1
  #for word in words:
    #print("word: " + word)
    
      
  print("total count: " + str(total_gib_count))
  print("gib count: " + str(gibberish_count))
  if gibberish_count / total_gib_count < 0.25:
    gibberish = False
    
  return gibberish

def percent_gibberish(line):
  global gibberish
  global gibberish_count
  global total_gib_count
  words = line.split()
  print(pydict.meaning(line))
  if pydict.meaning(line) is None:
    gibberish_count+=1
    total_gib_count+=1
  else:
    total_gib_count+=1
  #for word in words:
    #print("word: " + word)
    
      
  print("total count: " + str(total_gib_count))
  print("gib count: " + str(gibberish_count))
  print (gibberish_count / total_gib_count)
  return gibberish_count / total_gib_count
  

In [0]:
def determine_mashing(line):
  # assumes line is gibberish
  # key set only includes the alphabet, not the other keys on the keyboard because i'm lazy
  common_keys = {"Q":["W","A","S"],"W":["Q","A","S","D","E"], "E":["W","S","D","F","R"], "R":["E","D","F","G","T"],"T":["R","F","G","H","Y"],
                "Y":["T","G","H","J","Y"], "U":["Y","H","J","I","K"], "I":["U","J","K","L","O"], "O":["I","K","L","P"],
                "A":["Q","W","S","Z"],"S":["Q","W","E","D","X","Z","A"],"D":["W","E","R","F","C","X","S"],"F":["D","E","R","T","G","V","C"],
                "G":["R","T","Y","H","B","V","F"],"H":["T","Y","U","J","N","B","G"],"J":["Y","U","I","K","M","N","H"],
                "K":["U","I","O","L","M","J"],"L":["I","O","P","K","M"],"Z":["A","S","X"],"X":["Z","S","D","C"],"C":["X","D","F","V"],
                "V":["C","F","G","B"],"B":["V","G","H","N"],"N":["B","H","J","M"],"M":["N","J","K","L",","]}
  
  mashing = False
  mash_count = 0
  total_count = 1
  words = str(line.split())
  letters = list(words)

  for word in words: # I think this might not work because it doesn't move on to another word until it checked the same thing a bunch of times
    if words.index(word) + 3 is not None and words.index(word) - 3 is not None:
      for letter in letters:   
        if ((letters.index(letter) in common_keys and (letters[letters.index(letter) + 1] in common_keys
                                                    and letters[letters.index(letter) + 2] in common_keys
                                                    and letters[letters.index(letter) + 3] in common_keys)) 
        or (letters.index(letter) in common_keys and (letters[letters.index(letter) - 1] in common_keys
                                                     and letters[letters.index(letter) - 2] in common_keys
                                                     and letters[letters.index(letter) - 3] in common_keys))):
          mash_count+=1
          total_count+=1
    elif words.index(word) + 2 is not None and words.index(word) - 2 is not None:
      for letter in letters:
        if ((letters.index(letter) in common_keys and (letters[letters.index(letter) + 1] in common_keys
                                                     and letters.index(letter) + 2 in common_keys))
        or (letters.index(letter) in common_keys and (letters.index(letter) - 1 in common_keys
                                                     and letters.index(letter) - 2 in common_keys))):
          mash_count+=1
          total_count+=1
    elif words.index(word) + 1 is not None and words.index(word) - 1 is not None:
      for letter in letters:
        if ((letters.index(letter) in common_keys and (letters.index(letter) + 1 in common_keys))
        or (letters.index(letter) in common_keys and (letters.index(letter) - 1 in common_keys))):
          mash_count+=1
          total_count+=1
    else:
      total_count+=1
      
  if (mash_count/total_count > 0.75):
    mashing = True
    
  return mashing
          

In [93]:
# Prompt the user for input for a file, examine the file line by line, and determine
# 1) whether or not it is gibberish, as well as the percent gibberish,
# and 2) if it is gibberish, the percent mashed



if __name__ == "__main__":
  gib_count = 0
  total_everything_count = 0
  mashing_count = 0
  total_mashing_count = 0
  
  f = input("Enter the full link to the file you wish to examine: ")
  data = urllib.urlopen(f)
  raw_data = data.read().decode('utf-8')
  spl = raw_data.split()
  
  for word in spl:
    if determine_gibberish(word):
        gib_count+=1
        total_everything_count+=1
    else:
      total_everything_count+=1

    if determine_mashing(word):
      mashing_count+=1
      total_mashing_count+=1
    else:
      total_mashing_count+=1
        
  calculated_gibberish = gib_count / total_everything_count
  calculated_mashing = (mashing_count / total_mashing_count)*100
  print("calculated gib: " + str(calculated_gibberish))
  print("total everything: " + str(total_everything_count))
  print("Your file is " + str(calculated_mashing) + "% gibberish")
    
  if calculated_gibberish >= 0.75 and calculated_mashing >= 0.75:
    print("Your file is determined to be gibberish and the result of keyboard mashing")
  elif calculated_gibberish >= 0.75 and calculated_mashing < 0.75:
    print("Your file is determined to be gibberish and the result of some other form of random generation -- not keyboard mashing")
  else:
    print("Your file is determined to not be gibberish")
  if percent_gibberish(raw_data) >= 0.75:
    print("Your file is gibberish")
  else:
    print("Your file is not gibberish")

Enter the full link to the file you wish to examine: https://raw.githubusercontent.com/theblindbandet/Random-Text/master/notgibberish.txt


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


{'Noun': ['an expression of greeting']}
total count: 20
gib count: 15
{'Noun': ['an expression of greeting']}
total count: 21
gib count: 15
Error: The Following Error occured: list index out of range
None
Error: The Following Error occured: list index out of range
total count: 22
gib count: 16
{'Noun': ['a nonmetallic element belonging to the halogens; used especially in medicine and photography and in dyes; occurs naturally only in combination in small quantities (as in sea water or rocks', 'the smallest whole number or a numeral representing this number', 'the 9th letter of the Roman alphabet'], 'Verb': ['have the quality of being; (copula, used with an adjective or a predicate noun', 'be identical to; be someone or something', 'occupy a certain position or area; be somewhere', 'have an existence, be extant', 'happen, occur, take place', 'be identical or equivalent to', 'form or compose', 'work in a specific place, with a specific subject, or in a specific function', 'represent, as o